In [ ]:
!pip install transformers
!pip install datasets
!pip install --upgrade --no-cache-dir gdown==4.5.4

!gdown 18oZZ4jqRK-uF-Nz6ftRdgNjKix88hrnO
!unzip data_and_models.zip && rm data_and_models.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.6 MB/s 
     |████████████████████████████████| 7.6 MB 72.2 MB/s 
     |████████████████████████████████| 163 kB 93.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.8 MB/s 
     |████████████████████████████████| 212 kB 63.8 MB/s 
     |████████████████████████████████| 115 kB 69.6 MB/s 
     |████████████████████████████████| 127 kB 70.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Archive:  data_and_models.zip
   creating

In [ ]:
import random
import time

from datasets import load_metric
import numpy as np
import pandas as pd
import pickle


def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

start = time.time()
directory = "./data_and_models/"
all_df = pd.read_csv(directory+"target_corpus.csv")

import torch
class PSCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

import sklearn
def top_k_accuracy(top_k, predictions, labels):
  assert len(predictions) == len(labels)
  total = 0
  correct = 0
  for i in range(len(predictions)):
    total += 1
    prediction = []
    for j, k in enumerate(predictions[i]):
      prediction.append([j, k]) # k is the value
    prediction.sort(key = lambda x: -x[1])
    for j, _ in prediction[:top_k]:
      if j == labels[i]:
        correct += 1
        break
  return correct/total

import transformers
from transformers import RobertaTokenizerFast
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')


metrics = ["Top-1 accuracy/F1 micro", "Top-3 accuracy", "Top-5 accuracy", "Balanced accuracy", "F1 macro"]
seeds = [11, 12, 13, 14, 15]
epochs = 20

tasks = {
    "44": {
        "number_of_labels": 42,
         "label_column": 1,
    },
    "8": {
        "number_of_labels": 8,
        "label_column": 2,
    }
}

def compute(task):
  t1, t3, t5, ba, f1 = [],[],[],[],[]
  baseline_t1, baseline_t3, baseline_t5, baseline_ba, baseline_f1 = [], [], [], [], []

  ###### load the cross-domain classifier
  with open(directory + "logistic_model_" + task + ".pkl", "rb") as doc:
          model = pickle.load(doc)
  with open(directory + "tfidf_" + task + ".pkl", "rb") as doc:
          cf_tokenizer = pickle.load(doc)
  class_mapper = {}
  class_reverse_mapper = {}
  for i, topic in enumerate(model.classes_):
      class_mapper[topic.replace(" ", ".").replace("-", ".")] = i
      class_reverse_mapper[i] = topic

  for seed in seeds:
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)

    import csv
    from sklearn.model_selection import train_test_split

    index = -1
    classes = {}
    texts = []
    labels = []
    lm_reverse_mapper = {}
    with open(directory + "target_corpus.csv") as doc:
      reader = csv.reader(doc)
      next(reader)
      for row in reader:
        topic = row[tasks[task]["label_column"]]
        if topic not in classes:
          index += 1
          classes[topic] = index
          lm_reverse_mapper[index] = topic.capitalize()
        labels.append(classes[topic])
        texts.append(row[0])
    print("# classes", len(classes))
    X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=625, random_state=seed)
    X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=625, random_state=seed)
    print(len(X_train), len(X_dev), len(X_test))
    print("# classes in train", len(set(y_train)))
    print("# classes in dev", len(set(y_dev)))
    print("# classes in test", len(set(y_test)))

    mlength = 512
    train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=mlength)
    dev_encodings = tokenizer(X_dev, truncation=True, padding=True, max_length = mlength)
    test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length= mlength)


    train_dataset = PSCDataset(train_encodings, y_train)
    dev_dataset = PSCDataset(dev_encodings, y_dev)
    test_dataset = PSCDataset(test_encodings, y_test)

    training_args = TrainingArguments(
        output_dir="./results",          # output directory
        num_train_epochs=epochs,         # total number of training epochs
        per_device_train_batch_size=16,  # batch size per device during training
        per_device_eval_batch_size=64,   # batch size for evaluation
        warmup_steps=0,                  # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
        learning_rate = 2e-5,
        save_strategy= "epoch",
        evaluation_strategy="epoch",
        load_best_model_at_end= True,
        seed = seed, 
    )

    def model_init():
        return RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=tasks[task]["number_of_labels"])
    trainer = Trainer(
        model_init=model_init,               # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=dev_dataset,            # evaluation dataset
        compute_metrics=compute_metrics,     # compute_metrics
        )

    trainer.train()
    predictions = trainer.predict(test_dataset)
    preds = np.argmax(predictions.predictions, axis=-1)

    t1.append(top_k_accuracy(1, predictions.predictions, test_dataset.labels))
    t3.append(top_k_accuracy(3, predictions.predictions, test_dataset.labels))
    t5.append(top_k_accuracy(5, predictions.predictions, test_dataset.labels))
    ba.append(sklearn.metrics.balanced_accuracy_score(test_dataset.labels, preds))
    f1.append(sklearn.metrics.f1_score(test_dataset.labels, preds, average = "macro"))

    df = all_df[all_df["text"].isin(X_test)]
    X = df['text']
    Y = list(df["topic_"+ task].transform(lambda x: class_mapper[x]))

    Xtfidf = cf_tokenizer.transform(X)

    preds = model.predict(Xtfidf)
    preds = [class_mapper[topic.replace(" ", ".").replace("-", ".")] for topic in preds]
    policy_probs = model.predict_proba(Xtfidf)
    
    baseline_t1.append(top_k_accuracy(1, policy_probs, Y))
    baseline_t3.append(top_k_accuracy(3, policy_probs, Y))
    baseline_t5.append(top_k_accuracy(5, policy_probs, Y))
    baseline_ba.append(sklearn.metrics.balanced_accuracy_score(Y, preds))
    baseline_f1.append(sklearn.metrics.f1_score(Y, preds, average = "macro"))

  result = {}
  for metric, baseline, experiment in zip(metrics, [baseline_t1, baseline_t3, baseline_t5, baseline_ba, baseline_f1], [t1, t3, t5, ba, f1]):
    result[metric] = [np.mean(baseline), np.std(baseline), np.mean(experiment), np.std(baseline)]
  return result

results = {}
for task in tasks:
  result = compute(task)
  results[task] = result

np.save( directory + "table_1_results.npy", results)


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

# classes 42
2915 625 625
# classes in train 42
# classes in dev 36
# classes in test 35


...

***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-183
Configuration saved in ./results/checkpoint-183/config.json
Model weights saved in ./results/checkpoint-183/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-366
Configuration saved in ./results/checkpoint-366/config.json
Model weights saved in ./results/checkpoint-366/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-549
Configuration saved in ./results/checkpoint-549/config.json
Model weights saved in ./results/checkpoint-549/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-732
Configuration saved in ./results/checkpoint-732/config.json
Model weights saved in ./results/checkpoint-732/pytorch_model.bin
***** Ru

# classes 8
2915 625 625
# classes in train 8
# classes in dev 8
# classes in test 8


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,

Epoch,Training Loss,Validation Loss,Accuracy
1,1.292900,1.187130,0.603200
2,1.051200,1.085045,0.633600
3,0.789500,1.104119,0.632000
4,0.691700,1.240097,0.606400
5,0.346900,1.456409,0.614400
6,0.258400,1.631499,0.635200
7,0.291700,1.943097,0.614400
8,0.063100,2.069870,0.630400
9,0.150900,2.323310,0.609600
10,0.007000,2.446229,0.612800


***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-183
Configuration saved in ./results/checkpoint-183/config.json
Model weights saved in ./results/checkpoint-183/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-366
Configuration saved in ./results/checkpoint-366/config.json
Model weights saved in ./results/checkpoint-366/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-549
Configuration saved in ./results/checkpoint-549/config.json
Model weights saved in ./results/checkpoint-549/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-732
Configuration saved in ./results/checkpoint-732/config.json
Model weights saved in ./results/checkpoint-732/pytorch_model.bin
***** Ru

In [ ]:
def preprocess_result(result):
  output = []
  outperform = result[2] > result[0]
  for i, j in enumerate(result):
    j = str(round(j, 3))
    if len(j) < 5:
      j += "0" * (5-len(j))
    if i % 2 == 1: # standard deviation
      j = "(" + j + ")"
      if i == 1:
        j += " & "
    if outperform:
      if i == 2:
        j = "\\textbf{" + j
      if i == 3:
        j += "}"
    else:
      if i == 0:
        j = "\\textbf{" + j
      if i == 1:
        j += "}"
    output.append(j)
  return " ".join(output)

for metric in metrics:
  output = [metric]
  for task in tasks:
    output.append(preprocess_result(results[task][metric]))
  print(" & ".join(output) + "\\\\")

Top-1 accuracy/F1 micro & 0.414 (0.009) &  \textbf{0.527 (0.009)} & 0.515 (0.006) &  \textbf{0.631 (0.006)}\\
Top-3 accuracy & 0.656 (0.008) &  \textbf{0.744 (0.008)} & 0.819 (0.003) &  \textbf{0.904 (0.003)}\\
Top-5 accuracy & 0.752 (0.004) &  \textbf{0.828 (0.004)} & 0.921 (0.008) &  \textbf{0.969 (0.008)}\\
Balanced accuracy & 0.309 (0.030) &  \textbf{0.357 (0.030)} & 0.454 (0.014) &  \textbf{0.580 (0.014)}\\
F1 macro & 0.294 (0.025) &  \textbf{0.328 (0.025)} & 0.449 (0.014) &  \textbf{0.574 (0.014)}\\


In [ ]:
end = time.time()
print(f"The program took {(end - start) // 60} minutes in total.")

The program took 262.0 minutes in total.


In [ ]:
from google.colab import runtime
runtime.unassign()